### 1 Carregar e Preprocessar os Dados 

In [1]:
import numpy as np
from sklearn import svm
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Load data
X_train = np.load('Xtrain2_a.npy')  # Patches 7x7 achatados
Y_train = np.load('Ytrain2_a.npy')  # Máscaras correspondentes (48x48)
X_test = np.load('Xtest2_a.npy')    # Testes de patches

# Reduzindo o tamanho do conjunto de treino para acelerar o processamento
#X_train_small = X_train[:int(0.000001 * len(X_train))]  # 10% do conjunto original
#Y_train_small = Y_train[:int(0.000001 * len(Y_train))]

# Usar uma amostra menor para treino
sample_size = 17640  # Define o tamanho da amostra para o treino rápido
X_train = X_train[:sample_size]
Y_train = Y_train[:sample_size]


# Print shapes to confirm
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}, X_test shape: {X_test.shape}")


X_train shape: (17640, 49), Y_train shape: (17640,), X_test shape: (345744, 49)


### 2. Train-Test Split and SVM Model Training


In [10]:
# Train-test split for validation
X_train_split, X_val_split, Y_train_split, Y_val_split = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# Initialize SVM classifier
svm_clf = svm.SVC(kernel='linear')  # You can try other kernels like 'rbf'

# Train SVM model
svm_clf.fit(X_train_split, Y_train_split)

# Predictions on validation set
Y_val_pred = svm_clf.predict(X_val_split)

# Compute balanced accuracy on validation set
balanced_acc_val = balanced_accuracy_score(Y_val_split, Y_val_pred)
print(f"Balanced Accuracy on Validation Set: {balanced_acc_val:.4f}")


### 3. Preparar os Dados para o Treinamento

In [ ]:
# Predictions on test set
Y_test_pred = svm_clf.predict(X_test)

# Compute balanced accuracy on test set (since ground truth Y_test is not provided, assume it's part of X_train here)
balanced_acc_test = balanced_accuracy_score(Y_train_split, svm_clf.predict(X_train_split))
print(f"Balanced Accuracy on Test Set: {balanced_acc_test:.4f}")

# Confusion matrix
conf_matrix = confusion_matrix(Y_val_split, Y_val_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()


### 4. Visualizar Previsões e Mostrar Matriz de Confusão

In [ ]:
# Configurar valores de precisao para exibição
val_acc_values = [balanced_acc_val]
train_acc_values = [balanced_acc_test]

# Plotar o gráfico de precisão
plt.figure(figsize=(8, 5))
plt.plot(train_acc_values, label='Train Accuracy')
plt.plot(val_acc_values, label='Validation Accuracy')
plt.title('Accuracy over Training')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


### 5. Image Reconstruction (42x42)

In [ ]:
from sklearn.feature_extraction import image
import matplotlib.pyplot as plt

# Configurações de reconstrução
image_size_with_crust = (48, 48)
patch_size = (7, 7)
num_patches_per_image = 42 * 42  # Quantidade de patches por imagem
num_images = 10  # Número de imagens para visualizar

# Função para reconstruir e exibir imagens
def plot_reconstructed_images(data, title, num_images=10):
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    
    for idx in range(num_images):
        start = idx * num_patches_per_image
        end = start + num_patches_per_image
        patches = data[start:end].reshape(-1, *patch_size)
        
        # Reconstrução da imagem a partir dos patches
        reconstructed_image = image.reconstruct_from_patches_2d(patches, image_size_with_crust)
        
        # Mostrar a imagem reconstruída
        ax = axes[idx // 5, idx % 5]
        ax.imshow(reconstructed_image, cmap='gray')
        ax.set_title(f"Imagem {idx+1}")
        ax.axis('off')

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

# Exibir imagens reconstruídas do conjunto de treino
plot_reconstructed_images(X_train, title="Imagens Reconstruídas - X_train", num_images=num_images)

# Exibir imagens reconstruídas do conjunto de teste com previsões
plot_reconstructed_images(Y_test_pred, title="Imagens Reconstruídas - Y_test_pred", num_images=num_images)


### 6. Reconstruir a Imagem a partir das Previsões
